In [ ]:
!pip install -q crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 814.5/814.5 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
!pip install -q 'crewai[tools]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 2.1 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade -q crewai instructor

In [ ]:
import os

from crewai import Agent,Task, Crew, Process
from crewai_tools import SerperDevTool

from langchain.agents import load_tools
from langchain_openai import ChatOpenAI

api_key="sk-proj-9WrNxL8mwGR71Nsg5dlPT3BlbkFJnK22yo3Hw5SraFohJ2DF"
os.environ["OPENAI_API_KEY"]=api_key
os.environ["SERPER_API_KEY"] = "Y"

### Loading llm

In [ ]:
llm=ChatOpenAI(temperature=0,model="gpt-4-1106-preview")
llm_vision= ChatOpenAI(temperature=0, model="gpt-4-vision-preview", max_tokens=1024)

### Construct Tool

In [ ]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()

## Creating Prompt Template

In [ ]:
input_data = {
    "prompts": [
        ["what is happening in this image?", ["/content/Word Embeddings.png"]],
        #["give me an essay related to this image", ["/content/WhatsApp Image 2024-06-27 at 8.21.50 PM.jpeg"]],
        ["also explain me its importance with respect to NLP"]
    ],
    "output_format": "JSON",
    "example_output_structure": "{pic1: [fruit1, colorA] , pic2: [fruit2, colorX], pic3: [fruit3, colorY]}",
    "max_iterations": 5
}

In [ ]:
img_path="/content/Word Embeddings.png"

In [ ]:
output_format=input_data['output_format']

## Extracting the prompts

In [ ]:
# Function to parse input_data and extract prompts
def parse_input(input_data):
    prompts = []
    for item in input_data['prompts']:
        text_prompt = None
        image_path = None
        for element in item:
            if isinstance(element, str):
                text_prompt = element
            elif isinstance(element, list) and element and isinstance(element[0], str):
                image_path = element[0]
        prompts.append((text_prompt, image_path))
    return prompts

prompts=parse_input(input_data)

## Creating a Moderator

In [ ]:
# Define the manager agent
moderator = Agent(
    role="Manager",
    goal="Manage the crew and ensure the tasks are completed efficiently.",
    backstory="You're an experienced manager, skilled in overseeing complex projects and guiding teams to success. Your role is to coordinate the efforts of the crew members, ensuring that each task is completed on time and to the highest standard."
    f"the agents must interact and if one fails to give output, other should have a conversational text and the output of each agent should be mainatned in format{output_format}",
    allow_delegation=False,
    #prompt_template=input_data,
    #response_template=input_data['example_output_structure'],
)

### Construct Agents

In [ ]:
ImageAnalyserAgent = Agent(
  role='Image Analyser Agent',
  goal= f"You will use the provided tool to analyze image in the path {img_path} and respond to queries about them given in {prompts} and Remember to maintain the output schema mentioned in the {input_data}"
    f"Return the correct output format to the supervisor. Format is {output_format}"
    "You are working in a multi agent environment so collaborate with the other agent as well",
  backstory="""You are a an experienced employee in understanding the contents of an image and responding to any questions by user.
  You are an intelligent agent specializing in image analysis."""
  f"Return the correct output format to the supervisor. Format is {output_format}",

  verbose=True,
  allow_delegation=False,
  #system_template=,
  #prompt_template=input_data,
  #response_template=input_data['example_output_structure'],
  #max_rpm=, # max number of requests per minute
  max_iter=5, #max number of iterations of the agent
  llm=llm_vision,
)
ConversationAgent = Agent(
  role='Conveersation Q/A Agent',
  goal=f"Answer questions by the user {prompts} and be conversational"
   f"You will use the provided tool to analyze text and respond to queries about them and Remember to maintain the output format {output_format}",
  backstory=""" You are an intelligent agent specializing in text analysis and conversation to solve user's problems"
    "You are working in a multi agent environment so collaborate with the other agent as well.""",
  verbose=True,
  allow_delegation=True,
  #system_template=,
  #prompt_template=input_data,
  #response_template=input_data['example_output_structure'],
  llm=llm
)

## Defining Tasks for agents

In [ ]:
# Create tasks for your agents
task1 = Task(
  description="""See the image studey it and analyse it. Also work with the ConversationAgent to solve user's questions""",
  expected_output=f"The output should be in lines with the format user has given in {output_format}",
  agent=ImageAnalyserAgent
)

task2 = Task(
  description="""To understand the prompts sent by the user and have a conversation to solve the question.""",
  expected_output=f"The output should be in lines with the format user has given in {output_format}",
  agent=ConversationAgent
)

## Creating type of process

- Sequential
- Hierarchial



## Input Schema:


In [ ]:
input_schema = {
  "type": "object",
  "properties": {
    "prompts": {
      "type": "array",
      "items": {
        "type": "array",
        "items": {"type": ["string","array"]}
      }
    },
    "output_format": {
      "type": "string",
      "enum": ["list", "dict", "JSON"]
    },
    "example_output_structure": {"type": "string"},
    "max_iterations": {"type": "integer"}
  },
  "required": ["prompts", "output_format", "example_output_structure", "max_iterations"]
}

## Input Data

- Enter the ImagePath as list

In [ ]:
input_data = {
    "prompts": [
        ["what is happening in this image?", ["/content/Word Embeddings.png"]],
        #["give me an essay related to this image", ["/content/WhatsApp Image 2024-06-27 at 8.21.50 PM.jpeg"]],
        ["also i want to become like the person in this image what should i do?"]
    ],
    "output_format": "JSON",
    "example_output_structure": "{pic1: [fruit1, colorA] , pic2: [fruit2, colorX], pic3: [fruit3, colorY]}",
    "max_iterations": 5
}


## Validating the input data

In [ ]:
import jsonschema
from jsonschema import validate
import json

# Function to validate input
def validate_input(input_data):
    try:
        validate(instance=input_data, schema=input_schema)
    except jsonschema.exceptions.ValidationError as e:
        return False, str(e)
    return True, ""

In [ ]:
validate_input(input_data)

(True, '')

In [ ]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[ImageAnalyserAgent,ConversationAgent],
  tasks=[task1,task2],
  verbose=2, # You can set it to 1 or 2 to different logging levels
  #manager_llm=ChatOpenAI(temperature=0, model="gpt-4"),  # Mandatory if manager_agent is not set
  process=Process.hierarchical,  # Specifies the hierarchical management approach
  memory=True,
  manager_agent=moderator
)

In [ ]:
# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)

 [2024-06-29 09:27:56][DEBUG]: Working Agent: Manager
 [2024-06-29 09:27:56][INFO]: Starting Task: See the image studey it and analyse it. Also work with the ConversationAgent to solve user's questions


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
{
  "Word_Embeddings.png": [
    {
      "description": "The image depicts a diagram illustrating the concept of word embeddings in natural language processing (NLP). There are several circles representing different words, with lines connecting them to indicate relationships. The words are positioned in a way that shows clustering of similar words based on context and meaning.",
      "objects": ["circles", "lines", "words"],
      "colors": ["various colors representing different clusters"],
      "context": "Word embeddings are a type of word representation that allows words with similar meaning to have a similar representation. They are a crucial part of NLP as they help in capturing the context o